In [1]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import dotenv
import pendulum
from sqlalchemy import create_engine, asc, desc, or_
from sqlalchemy.orm import sessionmaker
from config import Settings
from models import MessageSql
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates
from datetime import timedelta
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

settings = Settings(_env_file=dotenv.find_dotenv())
valid_password = settings.thermostat_api_key.get_secret_value()
engine = create_engine(settings.db_url.get_secret_value())
Session = sessionmaker(bind=engine)

session = Session()
timezone = "America/New_York"
start = pendulum.datetime(2024, 10, 31, 0, 0, tz=timezone)
end = pendulum.datetime(2024, 10, 31, 23, 30, tz=timezone)
start_ms = int(start.timestamp() * 1000)
end_ms = int(end.timestamp() * 1000)

messages = session.query(MessageSql).filter(
    MessageSql.from_alias.like(f'%oak%'),
    or_(
        MessageSql.message_type_name == "batched.readings",
        MessageSql.message_type_name == "report"
        ),
    MessageSql.message_persisted_ms >= start_ms,
    MessageSql.message_persisted_ms <= end_ms,
).order_by(asc(MessageSql.message_persisted_ms)).all()

print(len(messages))

269


In [2]:
selected_channels = [
    'hp-lwt', 'hp-ewt', 'hp-odu-pwr', 'hp-idu-pwr', 'primary-pump-pwr', 
    'dist-swt', 'dist-rwt', 'zone_heat_calls',
    'store-hot-pipe', 'store-cold-pipe', 'store-pump-pwr',
    'buffer-cold-pipe', 'buffer-hot-pipe', 'dist-pump-pwr', 'buffer-depths', 'storage-depths']

In [3]:
channels = {}
for message in messages:
    for channel in message.payload['ChannelReadingList']:
        # Find the channel name
        if message.message_type_name == 'report':
            channel_name = channel['ChannelName']
        elif message.message_type_name == 'batched.readings':
            for dc in message.payload['DataChannelList']:
                if dc['Id'] == channel['ChannelId']:
                    channel_name = dc['Name']
        # Store the values and times for the channel
        if channel_name not in channels:
            channels[channel_name] = {
                'values': channel['ValueList'],
                'times': channel['ScadaReadTimeUnixMsList']
            }
        else:
            channels[channel_name]['values'].extend(channel['ValueList'])
            channels[channel_name]['times'].extend(channel['ScadaReadTimeUnixMsList'])

# Sort values according to time
for key in channels.keys():
    sorted_times_values = sorted(zip(channels[key]['times'], channels[key]['values']))
    sorted_times, sorted_values = zip(*sorted_times_values)
    channels[key]['values'] = list(sorted_values)
    channels[key]['times'] = pd.to_datetime(list(sorted_times), unit='ms', utc=True)
    channels[key]['times'] = channels[key]['times'].tz_convert('America/New_York')
    channels[key]['times'] = [x.replace(tzinfo=None) for x in channels[key]['times']]

    # Check the length
    if len(channels[key]['times']) != len(channels[key]['values']):
        print(f"Length mismatch in channel: {key}")
        selected_channels.remove(key)
            
# Find all zone channels
zones = {}
first_times, process_heatcalls = None, False
for channel_name in channels.keys():
    if 'zone' in channel_name and 'gw-temp' not in channel_name:
        if 'state' not in channel_name:
            channels[channel_name]['values'] = [x/1000 for x in channels[channel_name]['values']]
        else:
            # Round times to the minute
            channels[channel_name]['times'] = pd.Series(channels[channel_name]['times']).dt.round('s').tolist()
            if first_times is None:
                first_times = channels[channel_name]['times']
            if channels[channel_name]['times'] != first_times:
                process_heatcalls = True
        zone_name = channel_name.split('-')[0]
        if zone_name not in zones:
            zones[zone_name] = [channel_name]
        else:
            zones[zone_name].append(channel_name)

if process_heatcalls:

    # Get all timestamps in the zone states
    all_times = []
    for zone in zones:
        for state in [x for x in zones[zone] if 'state' in x]:
            all_times.extend(channels[state]['times'])
    all_times = sorted(list(set(all_times)))

    channels_copy = channels.copy()
    for zone in zones:
        for state in [x for x in zones[zone] if 'state' in x]:
            if channels[state]['times'] != all_times:
                # Remove duplicates
                unique_dict = {}
                for time, value in zip(channels[state]['times'], channels[state]['values']):
                    if time not in unique_dict:
                        unique_dict[time] = value
                channels[state]['times'] = list(unique_dict.keys())
                channels[state]['values'] = list(unique_dict.values())
                # Add missing times
                for time in all_times:
                    if time not in channels[state]['times']:
                        channels[state]['times'].append(time)
                        channels[state]['values'].append(0)
                # Sort by time again
                sorted_times_values = sorted(zip(channels[state]['times'], channels[state]['values']))
                sorted_times, sorted_values = zip(*sorted_times_values)
                channels[state]['times'] = list(sorted_times)
                channels[state]['values'] = list(sorted_values)

In [ ]:
import pandas as pd
import numpy as np

csv_times = np.linspace(start_ms, end_ms, 3600)
csv_times_dt = [pd.to_datetime(x, unit='ms', utc=True) for x in csv_times]
csv_times_dt = [x.tz_convert('America/New_York').replace(tzinfo=None) for x in csv_times_dt]
csv_values = {}

for channel in channels:
    merged = pd.merge_asof(
        pd.DataFrame({'times': csv_times_dt}),
        pd.DataFrame(channels[channel]),
        on='times',
        direction='backward'
    )
    csv_values[channel] = list(merged['values'])

df = pd.DataFrame(csv_values)
df['timestamps'] = csv_times
df = df[['timestamps'] + [col for col in df.columns if col != 'timestamps']]
display(df.head())
df.to_csv(f'output_{time.time()}.csv')

,timestamps,primary-flow,primary-flow-hz,dist-flow,dist-flow-hz,hp-odu-pwr,hp-idu-pwr,dist-pump-pwr,primary-pump-pwr,store-pump-pwr,...,tank2-depth3-micro-v,tank2-depth4-micro-v,tank3-depth1,tank3-depth2,tank3-depth3,tank3-depth4,tank3-depth1-micro-v,tank3-depth2-micro-v,tank3-depth3-micro-v,tank3-depth4-micro-v
0,1.730347e+12,669,1472395,0,0,4596,3804,0,13,0,...,2086726,2091054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.730347e+12,666,1485728,0,0,4596,3804,0,13,0,...,2086851,2091248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.730347e+12,654,1458054,0,0,36,48,0,13,0,...,2086851,2091820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.730347e+12,660,1471598,0,0,36,48,0,13,0,...,2086705,2090097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.730347e+12,0,0,0,0,36,48,0,0,0,...,2086705,2092027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_value_at_time(channel, csv_time):

    csv_time = pd.to_datetime(csv_time, unit='ms', utc=True)
    csv_time = csv_time.tz_convert('America/New_York').replace(tzinfo=None)

    # Find the value just before the given csv_time
    values_before = [channel['values'][i] 
                  for i in range(len(channel['values'])) 
                  if channel['times'][i] <= csv_time]
    if values_before:
        last_value = values_before[-1]
    else:
        last_value = np.nan
    return last_value

csv_times = np.linspace(start_ms, end_ms, 3600)
csv_values = {}

for channel in channels.keys():
    print(channel)
    csv_values_channel = []
    for csv_time in csv_times:
        csv_value = get_value_at_time(channels[channel], csv_time)
        csv_values_channel.append(csv_value)
    csv_values[channel] = csv_values_channel
